In [2]:
!nvidia-smi

Sat Jul  1 15:09:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install optuna
!pip install pytorch-tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [5]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier

In [6]:
import pandas as pd
import numpy as np
import os

chart_df = pd.read_csv("/content/drive/MyDrive/chart_with_sentiment_scores.csv")

In [10]:
# add technical indicators
import pandas_ta as ta
from tqdm.auto import tqdm
hours, days, months = [], [], []
for dt in tqdm(chart_df["datetime"]):
  dtobj = pd.to_datetime(dt)
  hours.append(dtobj.hour)
  days.append(dtobj.day)
  months.append(dtobj.month)

chart_df["hours"] = hours
chart_df["days"] = days
chart_df["months"] = months

chart_df["ebsw"] = chart_df.ta.ebsw(lookahead=False)
chart_df["cmf"] = chart_df.ta.cmf(lookahead=False)
chart_df["bop"] = chart_df.ta.bop(lookahead=False)
chart_df["rsi/100"] = chart_df.ta.rsi(lookahead=False) / 100.0
chart_df["hwma"] = chart_df.ta.hwma(lookahead=False)
chart_df["linreg"] = chart_df.ta.linreg(lookahead=False)
chart_df["hwma/close"] = chart_df["hwma"] / chart_df["close"]
chart_df["linreg/close"] = chart_df["linreg"] / chart_df["close"]
chart_df["high/low"] = chart_df["high"] / chart_df["low"]
chart_df["high/open"] = chart_df["high"] / chart_df["open"]
chart_df["low/open"] = chart_df["low"] / chart_df["open"]
chart_df["close/open"] = chart_df["close"] / chart_df["open"]
chart_df["high/close"] = chart_df["high"] / chart_df["close"]
chart_df["low/close"]  = chart_df["low"] / chart_df["close"]

for l in range(1, 6):
  for col in ["open", "high", "low", "close", "volume"]:
    val = chart_df[col].values
    val_ret = [None for _ in range(l)]
    for i in range(l, len(val)):
      if val[i-l] == 0:
        ret = 1
      else:
        ret = val[i] / val[i-l]
      val_ret.append(ret)
    chart_df[f"{col}_change_{l}"] = val_ret

chart_df = chart_df.drop(columns={"datetime", "open", "high", "low", "close", "volume", "linreg", "hwma"})

  0%|          | 0/9335 [00:00<?, ?it/s]

In [11]:
chart_df.dropna(inplace=True)

In [12]:
chart_df.head()

,targets,news_positive_scores,news_negative_scores,hours,days,months,ebsw,cmf,bop,rsi/100,...,open_change_4,high_change_4,low_change_4,close_change_4,volume_change_4,open_change_5,high_change_5,low_change_5,close_change_5,volume_change_5
39,2.0,0.000000,0.000000,4,1,4,0.000000,0.221446,-0.045573,0.659537,...,1.004453,1.007434,1.006192,1.007774,1.143676,1.010897,1.006783,1.008813,1.004198,1.477008
40,2.0,0.000000,0.000000,8,1,4,0.577350,0.201553,-0.005917,0.658919,...,1.007774,1.006996,1.003691,1.005484,1.730719,1.004198,1.006005,1.003780,1.007735,1.223106
41,2.0,13.904989,8.095011,12,1,4,0.796874,0.172388,0.514761,0.690842,...,1.005480,1.011357,1.006952,1.010020,3.140040,1.007732,1.011994,1.007534,1.009474,2.201117
42,2.0,8.305100,3.694901,16,1,4,0.983671,0.021719,-0.328696,0.659900,...,1.010377,1.003634,1.016228,1.001828,0.437554,1.009456,1.010700,1.008450,1.008163,2.220246
43,0.0,8.310017,6.689983,20,1,4,0.986626,0.075315,0.291923,0.674547,...,1.002053,1.002638,1.003106,1.003805,0.600674,1.008761,1.003386,1.016425,1.003550,0.273292


In [13]:
train_columns = []

for col in chart_df.columns:
  if col != "targets":
    train_columns.append(col)

X = chart_df[train_columns]
Y = chart_df["targets"]

In [14]:
X.shape, Y.shape

((9296, 42), (9296,))

In [15]:
import warnings

# Disable all user warnings
warnings.filterwarnings("ignore")

In [ ]:
def objective(trial):
  n_d = trial.suggest_categorical("n_d", [8, 16, 32])
  n_a = trial.suggest_categorical("n_a", [8, 16, 32])
  n_steps = trial.suggest_categorical("n_steps", [3, 4, 5])
  gamma = trial.suggest_float("gamma", 1.0, 2.0)
  mask_type = trial.suggest_categorical("mask_type", ["sparsemax", "entmax"])
  batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
  max_epochs = trial.suggest_int("max_epochs", 50, 200)
  tscv = TimeSeriesSplit(n_splits=5)
  accuracies = []
  for train_idx, val_idx in tscv.split(X):
    X_train, X_val = X.iloc[train_idx].values, X.iloc[val_idx].values
    Y_train, Y_val = Y.iloc[train_idx].values, Y.iloc[val_idx].values
    model = TabNetClassifier(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        optimizer_fn=optim.Adam,
        optimizer_params=dict(lr=1e-3),
        mask_type=mask_type,
        device_name="cuda" if torch.cuda.is_available() else "cpu",
        verbose=0 # do not print training logs
    )
    model.fit(
        X_train = X_train,
        y_train = Y_train,
        eval_set = [(X_val, Y_val)],
        eval_metric = ["balanced_accuracy"],
        patience = 100,
        batch_size=batch_size,
        virtual_batch_size = batch_size,
        num_workers = 0,
        max_epochs=max_epochs,
        drop_last = False
    )
    Y_pred = model.predict(X_val)
    accuracy = accuracy_score(Y_val, Y_pred)
    accuracies.append(accuracy)
  return sum(accuracies) / len(accuracies)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
best_params = study.best_params
print(f"Best Parameters: {best_params}")

[I 2023-07-01 15:22:26,170] A new study created in memory with name: no-name-2dcaf147-3f8e-43e4-b7be-e3cbfed4d012


Stop training because you reached max_epochs = 166 with best_epoch = 122 and best_val_0_balanced_accuracy = 0.4375
Stop training because you reached max_epochs = 166 with best_epoch = 86 and best_val_0_balanced_accuracy = 0.53553

Early stopping occurred at epoch 142 with best_epoch = 42 and best_val_0_balanced_accuracy = 0.39699
Stop training because you reached max_epochs = 166 with best_epoch = 158 and best_val_0_balanced_accuracy = 0.43695

Early stopping occurred at epoch 161 with best_epoch = 61 and best_val_0_balanced_accuracy = 0.45854


[I 2023-07-01 15:35:32,173] Trial 0 finished with value: 0.45668173014848285 and parameters: {'n_d': 32, 'n_a': 16, 'n_steps': 3, 'gamma': 1.5331732720751277, 'mask_type': 'entmax', 'batch_size': 128, 'max_epochs': 166}. Best is trial 0 with value: 0.45668173014848285.


Stop training because you reached max_epochs = 141 with best_epoch = 51 and best_val_0_balanced_accuracy = 0.44827
Stop training because you reached max_epochs = 141 with best_epoch = 110 and best_val_0_balanced_accuracy = 0.5451

Early stopping occurred at epoch 132 with best_epoch = 32 and best_val_0_balanced_accuracy = 0.40614
Stop training because you reached max_epochs = 141 with best_epoch = 134 and best_val_0_balanced_accuracy = 0.4515
Stop training because you reached max_epochs = 141 with best_epoch = 78 and best_val_0_balanced_accuracy = 0.46857


[I 2023-07-01 16:29:05,732] Trial 1 finished with value: 0.4565526145900581 and parameters: {'n_d': 32, 'n_a': 32, 'n_steps': 4, 'gamma': 1.923618395436067, 'mask_type': 'sparsemax', 'batch_size': 32, 'max_epochs': 141}. Best is trial 0 with value: 0.45668173014848285.


...

Stop training because you reached max_epochs = 147 with best_epoch = 129 and best_val_0_balanced_accuracy = 0.4668
Stop training because you reached max_epochs = 147 with best_epoch = 115 and best_val_0_balanced_accuracy = 0.55413
Stop training because you reached max_epochs = 147 with best_epoch = 119 and best_val_0_balanced_accuracy = 0.43235
Stop training because you reached max_epochs = 147 with best_epoch = 56 and best_val_0_balanced_accuracy = 0.4487
Stop training because you reached max_epochs = 147 with best_epoch = 118 and best_val_0_balanced_accuracy = 0.46553


[I 2023-07-02 05:04:00,490] Trial 20 finished with value: 0.4704970948999354 and parameters: {'n_d': 16, 'n_a': 16, 'n_steps': 3, 'gamma': 1.2551219672660037, 'mask_type': 'sparsemax', 'batch_size': 32, 'max_epochs': 147}. Best is trial 15 with value: 0.48160103292446743.


Stop training because you reached max_epochs = 184 with best_epoch = 136 and best_val_0_balanced_accuracy = 0.45113
Stop training because you reached max_epochs = 184 with best_epoch = 132 and best_val_0_balanced_accuracy = 0.56093

Early stopping occurred at epoch 126 with best_epoch = 26 and best_val_0_balanced_accuracy = 0.40581

Early stopping occurred at epoch 177 with best_epoch = 77 and best_val_0_balanced_accuracy = 0.44519
Stop training because you reached max_epochs = 184 with best_epoch = 94 and best_val_0_balanced_accuracy = 0.48874


[I 2023-07-02 06:05:47,380] Trial 21 finished with value: 0.4605551969012266 and parameters: {'n_d': 16, 'n_a': 16, 'n_steps': 3, 'gamma': 1.3705088397310103, 'mask_type': 'sparsemax', 'batch_size': 32, 'max_epochs': 184}. Best is trial 15 with value: 0.48160103292446743.


Stop training because you reached max_epochs = 187 with best_epoch = 105 and best_val_0_balanced_accuracy = 0.46467
Stop training because you reached max_epochs = 187 with best_epoch = 171 and best_val_0_balanced_accuracy = 0.54002

Early stopping occurred at epoch 119 with best_epoch = 19 and best_val_0_balanced_accuracy = 0.41751
Stop training because you reached max_epochs = 187 with best_epoch = 90 and best_val_0_balanced_accuracy = 0.45649
